In [1]:
"""Report the estimated disparity map and the calculated MSE with respect to the given ground truth disparity maps"""

#leftDisparity_MSE_3 47.9622263732
#rightDisparity_MSE_3 46.7877765454
#leftDisparity_MSE_9 36.5163504757
#rightDisparity_MSE_9 35.7310431382


'Report the estimated disparity map and the calculated MSE with respect to the given ground truth disparity maps'

In [2]:
import os
os.chdir('/Users/sp/Desktop/CVIP/PA2Data')
import cv2
import numpy as np

left_image = cv2.imread('view1.png', 0)
right_image = cv2.imread('view5.png', 0)
ground_truth_left_image = cv2.imread('disp1.png', 0)
ground_truth_right_image = cv2.imread('disp5.png', 0)
width, height = left_image.shape[:2]
leftDisparityMap = np.zeros([width,height], np.uint8)
rightDisparityMap = np.zeros([width,height], np.uint8)
max_disparity = 74

In [3]:
def leftDisparity(pad):
    paddedImage_left = cv2.copyMakeBorder(left_image, pad, pad,pad,pad,cv2.BORDER_CONSTANT, value= 0)
    paddedImage_right = cv2.copyMakeBorder(right_image,pad,pad,pad,pad,cv2.BORDER_CONSTANT, value=0)
    for i in range(pad,width - pad):
        for j in range(pad, height - pad):
            left_slice = paddedImage_left[i - pad: i+pad+1 , j - pad: j+pad+1]
            reset = pad-1
            match_index = 0
            min_ssd = np.iinfo.max    
            if(j > max_disparity+pad-1):
                reset = j - max_disparity #(current_left_pixel - disparity) to avoid overflow
            for k in range(j, reset, -1):
                right_slice = paddedImage_right[i - pad: i + pad+1, k - pad:k + pad + 1]
                current_ssd = np.sum(np.square(left_slice - right_slice))
                if((k == j) or (current_ssd < min_ssd)):
                    min_ssd = current_ssd
                    match_index = k #match_index represents best matching pixel in right image
            leftDisparityMap[i - pad, j - pad] = j - match_index #disparity
    mse = np.mean((leftDisparityMap - ground_truth_left_image)**2)
    print("leftDisparity_MSE_" +str(pad*2+1)+"_win="+ str(mse))
    cv2.imwrite(str(pad*2+1)+"_win_" + "left_disp.png", leftDisparityMap) 
    return leftDisparityMap

In [4]:
leftDisparity_3win = leftDisparity(1)

leftDisparity_MSE_3_win=47.9622263732


In [5]:
leftDisparity_9win = leftDisparity(4)

leftDisparity_MSE_9_win=35.1475920845


In [17]:
"""Disparity Computation for Right Image with 3X3 window"""
def rightDisparity(pad):
    paddedImage_left = cv2.copyMakeBorder(left_image, pad, pad,pad,pad,cv2.BORDER_CONSTANT, value= 0)
    paddedImage_right = cv2.copyMakeBorder(right_image,pad,pad,pad,pad,cv2.BORDER_CONSTANT, value=0)
    for i in range(pad, width + pad):
        for j in range(pad, height + pad):
            right_slice = paddedImage_right[i - pad:i + pad+1, j -pad :j + pad +1]
            reset = j + max_disparity
            match_index = 0
            min_ssd = np.iinfo.max 
            if(reset > height):
                reset = height 
            for k in range(j, reset):
                left_slice = paddedImage_left[i - pad: i + pad + 1, k - pad:k + pad +1]
                current_ssd = np.sum(np.square(right_slice - left_slice))
                if((k == j) or (current_ssd < min_ssd)):
                    min_ssd = current_ssd
                    match_index = k #match_index represents best matching pixel in right image
            rightDisparityMap[i - pad, j - pad] = match_index - j
    mse = np.mean((rightDisparityMap - ground_truth_right_image)**2)
    print("rightDisparity_MSE_" +str(pad*2+1)+"_win="+ str(mse))
    cv2.imwrite(str(pad*2+1)+"_win_"+"right_disp.png", rightDisparityMap)
    return rightDisparityMap

In [18]:
rightDisparity_9win = rightDisparity(4)

rightDisparity_MSE_9_win=34.0928550581


In [20]:
rightDisparity_3win = rightDisparity(1)

rightDisparity_MSE_3_win=46.4708598447


In [9]:
#Consistency check

In [10]:
def MSE(consistent_win,ground_truth_image):  
    square_error = np.zeros([width, height], np.uint8)
    error = 0
    count = 0
    #calculate mse for consistent pixels
    for i in range(width):
        for j in range(height):
            if(consistent_win[i][j] != 0):
                square_error[i][j] = (consistent_win[i][j] - ground_truth_image[i][j])**2
                error += square_error[i][j]
                count += 1
    mse = error /count
    return mse

In [11]:
#3*3 left 

consistent_left_3_win = np.zeros([width,height], np.uint8)

#Compare disparity values
for i in range(0,width):
    for j in range(0,height):
        p=leftDisparity_3win[i][j]
        q=rightDisparity_3win[i][j]
        if(j - q >= 0 and p == rightDisparity_3win[i][j - p]):
            consistent_left_3_win[i][j] = p
mse = MSE(consistent_left_3_win,ground_truth_left_image)
print("consistent_left_3_win_mse = " + str(mse))
cv2.imwrite("consistent_left_3_win.png", consistent_left_3_win)

#9*9 left
consistent_left_9_win = np.zeros([width,height], np.uint8)

#form the consistent left disparity matrix by comparing the disparity values
for i in range(width):
    for j in range(height):
        p=leftDisparity_9win[i][j]
        q=rightDisparity_9win[i][j]
        if(j - p >= 0 and p == rightDisparity_9win[i][j - p]):
            consistent_left_9_win[i][j] = p

mse= MSE(consistent_left_9_win,ground_truth_left_image)
print("consistent_left_9_win_mse = " + str(mse))
cv2.imwrite("consistent_left_9_win.png", consistent_left_9_win)


/Users/sp/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: overflow encountered in ubyte_scalars
  if __name__ == '__main__':


consistent_left_3_win_mse = 6.93845757438
consistent_left_9_win_mse = 7.31207572055


True

In [21]:
#3*3 right
consistent_right_3_win = np.zeros([width,height], np.uint8)

#form the consistent right disparity matrix by comparing the disparity values
for i in range(0,width):
    for j in range(0,height):
        q=rightDisparity_3win[i][j]
        if(j + q < height and q == leftDisparity_3win[i][j + q]):
            consistent_right_3_win[i][j] = q
mse= MSE(consistent_right_3_win,ground_truth_right_image)
print("consistent_right_3_win_mse = " + str(mse))
cv2.imwrite("consistent_right_3_win.png", consistent_right_3_win)

#9*9 right
consistent_right_9_win = np.zeros([width,height], np.uint8)
for i in range(0,width):
    for j in range(0,height):
        q=rightDisparity_9win[i][j]
        if(j + q < height and q == leftDisparity_9win[i][j + q]):
            consistent_right_9_win[i][j] = q
mse2=MSE(consistent_right_9_win,ground_truth_right_image)
print("consistent_right_9_win_mse = " + str(mse2))
cv2.imwrite("consistent_right_9_win.png", consistent_right_9_win)

/Users/sp/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: overflow encountered in ubyte_scalars
  if __name__ == '__main__':


consistent_right_3_win_mse = 7.51090034524
consistent_right_9_win_mse = 7.51090034524


True

In [13]:
rightDisparity_3win


array([[32, 32, 31, ...,  1,  0,  0],
       [32, 28, 27, ...,  0,  0,  0],
       [28, 25, 24, ...,  0,  0,  0],
       ..., 
       [67, 73, 73, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0]], dtype=uint8)

In [16]:
sum(rightDisparity_9win-rightDisparity_3win)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0,